VGG19 - Transfer Learning Model Implementation - Stefano Maxenti - Riccardo Mencucci - "team_durian"

In [ ]:
!pip install --upgrade pip
!pip install scipy sklearn pandas seaborn pillow visualkeras
import tensorflow as tf
import numpy as np
import os
import random
import time
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [2]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
DATA_ROOT = "DATASET_AUG"

In [21]:
COLOR_MODE = 'rgb'
BATCH_SIZE = 128

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "DATASET_AUG/train"
valid_dir = "DATASET_AUG/val"

# rescaling makes lookup fail?
# no, casting issues.
from tensorflow.keras.applications.vgg19 import preprocess_input
train_data_generator = ImageDataGenerator(
                                            preprocessing_function=preprocess_input)

# keras onine augmentation drammatically increases training time
# dense training from < 1 min to > 5 min
valid_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_data_generator.flow_from_directory(
    directory = train_dir,
    # desired size for image, regardless of
    # image data. Images will be resized to fit
    # if necessary.
    # We use original size ==> no transfrom
    target_size = (256,256),
    # 3 color channel.
    # again, match data on disk.
    color_mode = COLOR_MODE,
    # default for classes is None.
    # That infers from name of folder classes the integer associated
    # with the class in lexicographical order.
    # This matches the order we used before.
    # We can change this by passing a list of strings to classes,
    # which should contain all (?) folder names. For now, the default one
    # will suffice.
    classes = None,
    # we need not specify this in model.fit, but we can do so here at data
    # generation.
    batch_size = BATCH_SIZE,
    shuffle = True
)

valid_gen = valid_data_generator.flow_from_directory(
    directory = valid_dir,
    target_size = (256,256),
    color_mode = COLOR_MODE,
    classes = None,
    batch_size = BATCH_SIZE,
    shuffle = True
)

Found 18541 images belonging to 14 classes.
Found 6187 images belonging to 14 classes.


In [ ]:
def get_next_batch(generator):
  batch = next(generator)

  image = batch[0]
  target = batch[1]

  print("(Input) image shape:", image.shape)
  print("Target shape:",target.shape)

  # Visualize only the first sample
  image = image[0]
  target = target[0]
  target_idx = np.argmax(target)
  print()
  print("Categorical label:", target)
  print("Label:", target_idx)
  #print("Class name:", labels[target_idx])
  fig = plt.figure(figsize=(6, 4))
  plt.imshow(np.uint8(image*255))

  return batch

_ = get_next_batch(valid_gen)

In [5]:
# Utility function to create folders and callbacks for training
from datetime import datetime
import io
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns


def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('data_augmentation_experiments_noaug')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_h%H_m%M_s%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tfk.callbacks.ModelCheckpoint(
      filepath=os.path.join(ckpt_dir, 'cp', "ckpt_{epoch:03d}_{val_loss:.5f}.hdf5"),
      save_weights_only=False,
      # if true, only save best epoch.
      # otherwise, save last epoch.
      # As we use early stopping, which already returns best model,
      # by setting false here he get both.

      save_best_only = True  
  )
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  tb_callback = tfk.callbacks.TensorBoard(
      log_dir = tb_dir,
      # avoid profiling to increase runtime performance
      profile_batch = 0,
      histogram_freq = 1
  )
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
  callbacks.append(es_callback)
  return callbacks

In [6]:
input_shape = (256,256,3)
epochs_dense = 10
epochs_finetuning = 125

In [7]:
# transfer learning, import VGG and build model


supernet = tfk.applications.VGG19(
    # remove classifier, only get fearure extractor
    include_top=False,
    weights="imagenet",
    input_shape=input_shape
)

supernet.trainable = False


inputs = tfk.Input(shape=input_shape)
x = supernet(inputs)
x = tfkl.GlobalAveragePooling2D(name='gpooling')(x)
#x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    128, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
    14, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

model = tfk.Model(inputs=inputs, outputs=outputs, name='model_transfer_learning_GAP')
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-3), metrics='accuracy')



for i, layer in enumerate(model.get_layer('vgg19').layers):
   print(i, layer.name, layer.trainable)
model.summary()

2021-11-28 09:53:21.092251: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 09:53:21.101527: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 09:53:21.102190: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 09:53:21.103711: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 09:53:21.104286: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read f

80150528/80134624 [==============================] - 3s 0us/step
0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_conv4 False
11 block3_pool False
12 block4_conv1 False
13 block4_conv2 False
14 block4_conv3 False
15 block4_conv4 False
16 block4_pool False
17 block5_conv1 False
18 block5_conv2 False
19 block5_conv3 False
20 block5_conv4 False
21 block5_pool False
Model: "model_transfer_learning_GAP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
vgg19 (Functional)           (None, 8, 8, 512)         20024384  
_________________________________________________________________
gpooling (GlobalAveragePooli (None, 51

In [8]:
# Create folders and callbacks and fit
#model = tfk.models.load_model("C:/Users/Riccardo/Downloads/10526141_10534455_2R/SubmissionModel")
model.summary()
aug_callbacks = create_folders_and_callbacks(model_name='transfer_gap')
print("starting at ", datetime.now())
# Train the model
history = model.fit(
    x = train_gen,
    epochs = epochs_dense,
    validation_data = valid_gen,
    callbacks = aug_callbacks,
).history

Model: "model_transfer_learning_GAP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
vgg19 (Functional)           (None, 8, 8, 512)         20024384  
_________________________________________________________________
gpooling (GlobalAveragePooli (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 14)

2021-11-28 09:53:29.192913: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2021-11-28 09:53:31.184686: I tensorflow/stream_executor/cuda/cuda_dnn.cc:381] Loaded cuDNN version 8204


  1/145 [..............................] - ETA: 37:22 - loss: 6.5321 - accuracy: 0.0938

2021-11-28 09:53:44.778162: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


145/145 [==============================] - 128s 784ms/step - loss: 1.6482 - accuracy: 0.5549 - val_loss: 0.4690 - val_accuracy: 0.8764
Epoch 2/10
145/145 [==============================] - 75s 519ms/step - loss: 0.6816 - accuracy: 0.7810 - val_loss: 0.2771 - val_accuracy: 0.9205
Epoch 3/10
145/145 [==============================] - 68s 470ms/step - loss: 0.5063 - accuracy: 0.8341 - val_loss: 0.2276 - val_accuracy: 0.9357
Epoch 4/10
145/145 [==============================] - 68s 471ms/step - loss: 0.4289 - accuracy: 0.8603 - val_loss: 0.1789 - val_accuracy: 0.9484
Epoch 5/10
145/145 [==============================] - 69s 472ms/step - loss: 0.3795 - accuracy: 0.8761 - val_loss: 0.1598 - val_accuracy: 0.9533
Epoch 6/10
145/145 [==============================] - 69s 472ms/step - loss: 0.3403 - accuracy: 0.8865 - val_loss: 0.1570 - val_accuracy: 0.9541
Epoch 7/10
145/145 [==============================] - 69s 472ms/step - loss: 0.3191 - accuracy: 0.8964 - val_loss: 0.1430 - val_accuracy: 0.

In [9]:
model_file_name = "model_dense_trained_final_noaug" + datetime.now().strftime('%b%d_h%H_m%M_s%S')
model.save(model_file_name)
print("model saved")

2021-11-28 10:07:25.232100: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model_dense_trained_final_noaugNov28_h10_m07_s24/assets
model saved


In [10]:
# make last classifier layer trainable for fine tuning
model.get_layer('vgg19').trainable = True

for i, layer in enumerate(model.get_layer('vgg19').layers[:11]):
   layer.trainable = False

for i, layer in enumerate(model.get_layer('vgg19').layers):
   print(i, layer.name, layer.trainable)
model.summary()

# lower learning rate
from keras import backend as K

print("Learning rate before update:", model.optimizer.learning_rate.numpy())

K.set_value(model.optimizer.learning_rate, 1e-4)

print("Learning rate after update:", model.optimizer.learning_rate.numpy())

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_conv4 False
11 block3_pool True
12 block4_conv1 True
13 block4_conv2 True
14 block4_conv3 True
15 block4_conv4 True
16 block4_pool True
17 block5_conv1 True
18 block5_conv2 True
19 block5_conv3 True
20 block5_conv4 True
21 block5_pool True
Model: "model_transfer_learning_GAP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
vgg19 (Functional)           (None, 8, 8, 512)         20024384  
_________________________________________________________________
gpooling (GlobalAveragePooli (None, 512)               0         
________________________________________________

In [11]:
history = model.fit(
    x = train_gen,
    initial_epoch = 10,
    epochs = epochs_finetuning,
    validation_data = valid_gen,
    callbacks = aug_callbacks,
    #class_weight = class_weights_dict
).history

Epoch 11/125
145/145 [==============================] - 65s 449ms/step - loss: 0.2419 - accuracy: 0.9207 - val_loss: 0.1159 - val_accuracy: 0.9670
Epoch 12/125
145/145 [==============================] - 68s 466ms/step - loss: 0.2398 - accuracy: 0.9195 - val_loss: 0.1147 - val_accuracy: 0.9683
Epoch 13/125
145/145 [==============================] - 67s 462ms/step - loss: 0.2338 - accuracy: 0.9208 - val_loss: 0.1133 - val_accuracy: 0.9669
Epoch 14/125
145/145 [==============================] - 68s 471ms/step - loss: 0.2401 - accuracy: 0.9216 - val_loss: 0.1133 - val_accuracy: 0.9675
Epoch 15/125
145/145 [==============================] - 68s 471ms/step - loss: 0.2318 - accuracy: 0.9229 - val_loss: 0.1116 - val_accuracy: 0.9678
Epoch 16/125
145/145 [==============================] - 69s 473ms/step - loss: 0.2339 - accuracy: 0.9223 - val_loss: 0.1116 - val_accuracy: 0.9678
Epoch 17/125
145/145 [==============================] - 69s 472ms/step - loss: 0.2299 - accuracy: 0.9240 - val_loss: 0

In [12]:
model_file_name = "model_finetune_trained_final_" + datetime.now().strftime('%b%d_h%H_m%M_s%S')
model.save(model_file_name)
print("model saved")
os.system(f"zip -r best_model_vgg19_final.zip ./{model_file_name}")

INFO:tensorflow:Assets written to: model_finetune_trained_final_Nov28_h12_m30_s23/assets
model saved
  adding: model_finetune_trained_final_Nov28_h12_m30_s23/ (stored 0%)
  adding: model_finetune_trained_final_Nov28_h12_m30_s23/assets/ (stored 0%)
  adding: model_finetune_trained_final_Nov28_h12_m30_s23/variables/ (stored 0%)
  adding: model_finetune_trained_final_Nov28_h12_m30_s23/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: model_finetune_trained_final_Nov28_h12_m30_s23/variables/variables.index (deflated 66%)
  adding: model_finetune_trained_final_Nov28_h12_m30_s23/saved_model.pb (deflated 90%)
  adding: model_finetune_trained_final_Nov28_h12_m30_s23/keras_metadata.pb (deflated 96%)


0

In [31]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

valid_gen.shuffle = False

Y_prediction = model.predict_generator(valid_gen, len(valid_gen))
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = valid_gen.classes
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
class_report = classification_report(Y_true, Y_pred_classes, 
                                     target_names=valid_gen.class_indices.keys())  # target_names must be ordered depending on the class labels
print('Confusion Matrix:')
print(confusion_mtx)
print()
print('Classification Report:')
print(class_report)

model.evaluate(valid_gen, return_dict=True)

/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[ 342    2    9    0    0    1    7    1    0    0    5    1    1    3]
 [   1  236    1    0    0    1    0    0    0    1    1    0    0    1]
 [   3    0  261    0    0    0    1    2    0    2    0    1    0    1]
 [   0    0    0  427    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0  484    0    0    0    0    1    0    0    4    1]
 [   0    0    1    0    0  558    1    1    0    0    1    0    0    0]
 [   2    0    0    0    0    1  363    1    0    0    0    1    0    2]
 [   3    2    0    0    0    1    1  304    0    0    3    0    1    2]
 [   2    0    0    0    0    0    1    1  286    0    5    0    0    9]
 [   0    0    0    0    0    0    0    0    0  190    0    0    1    0]
 [   0    1    1    0    0    2    0    2    2    0  520    0    0    1]
 [   0    0    0    1    0    0    0    0    0    0    0  266    0    2]
 [   0    0    0    0    0    0    0    0    1    5    0    0  288    0]
 [   3    1    0    1    1    4  

{'loss': 0.0819513201713562, 'accuracy': 0.9767253994941711}